In [2]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Small, MobileNetV3Large
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:
img_size = 224
batch_size = 64

train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    r'C:\Users\TAHMID HOSSAIN\Desktop\data\train',  # Replace with the path to your dataset
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    r'C:\Users\TAHMID HOSSAIN\Desktop\data\val',  # Replace with the path to your dataset
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 4571 images belonging to 4 classes.
Found 262 images belonging to 4 classes.


In [8]:
def create_mobilenetv3_model():
    base_model = MobileNetV3Large(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Freeze the base model for transfer learning

    # Add custom classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)  # Optional extra dense layer
    predictions = Dense(train_generator.num_classes, activation='softmax')(x)

    # Create the complete model
    model = Model(inputs=base_model.input, outputs=predictions)
    return model, base_model

# Unpack both model and base_model
model, base_model = create_mobilenetv3_model()



In [9]:
# Access base_model by name or index, depending on the setup
base_model = model.layers[0]
base_model.trainable = True



In [10]:
# Recompile with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune for a few additional epochs
history_fine_tune = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=15
)



Epoch 1/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.2642 - loss: 1.4457 - val_accuracy: 0.3906 - val_loss: 1.3765
Epoch 2/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.4219 - loss: 1.3205 - val_accuracy: 0.3333 - val_loss: 1.3759
Epoch 3/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 123s 2s/step - accuracy: 0.3924 - loss: 1.3063 - val_accuracy: 0.4102 - val_loss: 1.3303
Epoch 4/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4219 - loss: 1.2238 - val_accuracy: 0.6667 - val_loss: 1.3773
Epoch 5/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.4821 - loss: 1.2439 - val_accuracy: 0.5156 - val_loss: 1.3140
Epoch 6/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5312 - loss: 1.2131 - val_accuracy: 0.3333 - val_loss: 1.6621
Epoch 7/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.5321 - loss: 1.2049 - val_accuracy: 0.5430 - val_loss: 1.3117
Epoch 8/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5156 - loss: 1.2344 - val_accuracy: 0.1667 - val_

In [11]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')


base_model.trainable = True
# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Train again with fine-tuning
history_fine_tune = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10  # Fine-tune for a few additional epochs
)



5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.4928 - loss: 1.3284
Validation Loss: 1.316095232963562
Validation Accuracy: 0.5038167834281921
Epoch 1/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 150s 2s/step - accuracy: 0.5624 - loss: 1.1296 - val_accuracy: 0.4922 - val_loss: 1.3118
Epoch 2/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.5469 - loss: 1.1682 - val_accuracy: 0.1667 - val_loss: 1.5679
Epoch 3/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 123s 2s/step - accuracy: 0.6036 - loss: 1.1261 - val_accuracy: 0.5117 - val_loss: 1.3130
Epoch 4/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5312 - loss: 1.1408 - val_accuracy: 0.6667 - val_loss: 1.2403
Epoch 5/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - accuracy: 0.5906 - loss: 1.1053 - val_accuracy: 0.5078 - val_loss: 1.3056
Epoch 6/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5312 - loss: 1.2120 - val_accuracy: 0.5000 - val_loss: 1.3594
Epoch 7/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 128s 2s/step - accuracy: 0.6074 - loss: 1.0992 - val_accu